In [1]:
from os import chdir
from itertools import compress
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col, lit, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, CountVectorizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
from graphframes import *
import json
import re
import findspark
findspark.init()
print("spark ran")

chdir("C:\\Users\\chest\\Desktop\\MTech\\Big Data\\Project\\Data")

spark ran


In [2]:
#Experimentation of stopwords
stopwords=["packages", "ounces", "sheets", "sprig", "jars",\
                  "dashes", "ball", "jell", "tri", "grams", "shot",\
                  "cubes", "plastic", "jarred", "bunches", "prebaked",\
                  "can", "heads", "snickers", "bibb", "stemmed", "finely",\
                  "bunches", "trays", "spears", "free", "boston", "tablespoons",\
                  "pinches", "processed", "oz", "reynolds", "pouch", "bar", "carton",\
                  "solid", "inch", "inches", "cup", "cups", "gala", "thinly", "straight",\
                  "marinated", "drops", "one", "day", "buttery", "bing", "cube", "squeezes",\
                  "reduced", "double", "tubs", "bulbs", "sparkler", "scoop", "pint", "pouches",\
                  "long", "scoops", "shells", "comet", "shucked", "bars", "lean", "head", "hershey",\
                  "minature", "to", "hank", "cornish", "ear", "frilled", "curl", "frilled", "piece",\
                  "servings", "piece", "kaiser", "tenderflake", "dark", "servings", "quarts", "plain",\
                  "serving", "spiral", "skewers", "skewer", "navel", "quartered", "liter", "belgian", "bowl",\
                  "jigger", "o", "bottle", "portugese", "key", "foil", "thick", "roma", "birthday", "won", "slightly",\
                  "tablets", "raw", "bittersweet", "package", "jars", "weight", "teaspoons", "stalk", "unbaked", "feet",\
                  "mcintosh", "twist", "roasting", "canned", "tall", "recipes", "square", "sprigs", "slow", "envelopes",\
                  "warmed", "untreated", "ready", "archer", "inner", "packaged", "cans", "warm", "toothpick", "fresh", "fun", "bag",\
                  "hierloom", "bunch", "pkg", "strip", "unopened", "gallon", "ball", "skinned", "quart", "bouquet", "tubes", "vine", "sheet",\
                  "pieces", "splash", "container", "sliver", "purchased", "sliced", "racks", "half", "cubed", "wraps", "refrigerated", "sleeve", "london",\
                  "slices", "naval", "cleaned", "whisked", "pinch", "full", "sleeves", "pot", "metal", "sturdy", "muddled", "packets", "stalks", "prepared", "thin",\
                  "summer", "m", "large", "ring", "rolls", "box", "oven", "splashes", "plastic", "well", "loaves", "tiny", "sterilized", "market", "no", "sheets", "halves", "slice", "tablespoon",\
                  "slabs", "bricks", "pods", "gallons", "balls", "pre", "spot", "tub", "grilled", "torn", "canning", "slider", "brie", "cups", "or", "cubes", "pounds", "crumpets", "wedges", "crushed", "teaspoon",\
                  "individually", "sticks", "twists", "ears", "toothpicks", "links", "t", "cloves", "grinds", "trimmed", "drizzle", "tube", "shots", "organic", "boxes", "strips", "dash", "squirts", "diced", "beaten", "containers", "boneless",\
                  "skirt", "ounce", "hot", "unwrapped", "straw", "frozen", "roasted", "shredded", "dried"] 
#stale, jet, worm, fillets, multi
stopwords = list(set(stopwords))

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
print("Spark Session Created")
#Python Java issues: a drawback in using pyspark instead of Scala

Spark Session Created


In [4]:
path = "C:\\Users\\chest\\Desktop\\MTech\\Big Data\\Project\\Data\\allrecipes-recipes.json"
allrecipes_df = spark.read.json(path)
allrecipes_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- cook_time_minutes: long (nullable = true)
 |-- description: string (nullable = true)
 |-- error: boolean (nullable = true)
 |-- footnotes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- photo_url: string (nullable = true)
 |-- prep_time_minutes: long (nullable = true)
 |-- rating_stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- time_scraped: long (nullable = true)
 |-- title: string (nullable = true)
 |-- total_time_minutes: long (nullable = true)
 |-- url: string (nullable = true)



In [5]:
#create temporary view to transform to dataframes
allrecipes_df.createOrReplaceTempView("allrecipes")

#overall
allrecipes_df = spark.sql("SELECT title, description, ingredients, instructions, rating_stars, review_count, photo_url, total_time_minutes FROM allrecipes LIMIT 5000")
allrecipes_df.printSchema()
allrecipes_df.show(5)

#to generate into sample
# small_sample = allrecipes_df.toPandas()
# small_sample.to_csv("small_sample.csv")

#Noted cook_time_minutes have quite a bit of zeroes. Use rating_stars and review_count instead


root
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rating_stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- photo_url: string (nullable = true)
 |-- total_time_minutes: long (nullable = true)

+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+------------------+
|               title|         description|         ingredients|        instructions|rating_stars|review_count|           photo_url|total_time_minutes|
+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+------------------+
|Basil, Roasted Pe...|I just started ad...|[1/2 cup unsalted...|[Preheat oven to ...|        4.32|    

In [6]:
#User-defined Functions
def removeParenthesis(li):
    output = []
    for text in li:
        text = re.sub(r"\([^)]*\)", "", text)
        text = text.strip()
        output.append(text)
    return output
def removeNumbers(li):
    output = []
    for text in li:
        text = re.sub(r"\d+", "", text)
        text = re.sub(r"[//]", " ", text)
        text = text.strip()
        output.append(text)
    return output
def removeAfterComma(li):
    output = []
    for text in li:
        text = re.sub(r",[^,]*$", "", text)
        text = text.strip()
        output.append(text)
    return output
def removeStop(li, stoplist):
    output = []
    for text in li:
        text_tok = text.split(" ")
        text_output = [tok for tok in text_tok if tok not in stoplist]
        text_output = " ".join(text_output)
        output.append(text_output)
    return output

def convertString(li):
    output = []
    for text in li:
        text_combined = text.replace(" ", "-")
        output.append(text_combined)
    output_transformed = " ".join(output)
    return output_transformed

def convertString2(li):
    output = []
    for text in li:
        text_combined = text.replace(" ", "-")
        output.append(text_combined)
    return output

#create user defined function
udf_removeAfter = udf(removeAfterComma, ArrayType(StringType()))
udf_removeParen = udf(removeParenthesis, ArrayType(StringType()))
udf_removeNumbers = udf(removeNumbers, ArrayType(StringType()))
udf_removeStop = udf(lambda x: removeStop(x,stopwords), ArrayType(StringType()))
udf_convertString = udf(convertString, StringType())
udf_convertString2 = udf(convertString2, ArrayType(StringType()))

print("UDF Successful.")

UDF Successful.


In [30]:
#User Defined Function - Feature Engineering 
def detectVege(li, veg_list):
    label = []
    for text in li:
        if text not in veg_list:
            label.append(text)
            label.append("Veg")
    return label

#create user defined function
udf_detectVege = udf(lambda x: detectVege(x, veg_list), ArrayType(StringType()))


In [7]:
#create ingredient table
ingredient_df = allrecipes_df.select(["title", "ingredients"])
cleanIngredient_df = ingredient_df.withColumn('rm_paren', udf_removeParen('ingredients')).select('title','ingredients','rm_paren') \
                         .withColumn('rm_after', udf_removeAfter('rm_paren')).select('title','ingredients','rm_paren','rm_after') \
                         .withColumn('rm_numbers', udf_removeNumbers('rm_after')).select('title','ingredients','rm_paren','rm_after','rm_numbers') \
                         .withColumn('rm_complete', udf_removeStop('rm_numbers')).select('title','ingredients','rm_paren','rm_after','rm_numbers','rm_complete')
cleanIngredient_df.show(5)

#to convert into graph form
ingredient_graph = cleanIngredient_df.withColumn('exploded',explode('rm_complete')) \
                      .select(col('title').alias('Recipe'),col('exploded').alias('Ingredient'))\
                      .withColumn("Frequency", lit(1))
ingredient_graph = ingredient_graph.filter(ingredient_graph.Ingredient != "")
ingredient_graph.show(5)
#Let's try convert ingredient into count_frequency (BAD WAY, don't do)
# ingredient_freq = ingredient_graph.groupBy("Recipe").pivot("Ingredient").sum("Frequency")
# ingredient_freq.show()
# test = ingredient_freq.toPandas()
# test.to_csv("test.csv")
# print("test done")


#Proof of concept (Need to do !=)
# print(ingredient_graph.filter(ingredient_graph.Ingredient.isNull()).count())
# ingredient_graph = ingredient_graph.na.drop(subset=["Recipe"])
# test = ingredient_graph.filter(ingredient_graph.Recipe == "Grandma Emma's Spice Loaf")
# test.show()
# test2 = test.filter(ingredient_graph.Ingredient != "")
# test2.show()

# graph_sample = ingredient_graph.toPandas()
# graph_sample.to_csv("graph_sample.csv")

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|         ingredients|            rm_paren|            rm_after|          rm_numbers|         rm_complete|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Basil, Roasted Pe...|[1/2 cup unsalted...|[1/2 cup unsalted...|[1/2 cup unsalted...|[cup unsalted but...|[unsalted butter,...|
|Crispy Cheese Twists|[1/2 cup Parmesan...|[1/2 cup Parmesan...|[1/2 cup Parmesan...|[cup Parmesan che...|[Parmesan cheese,...|
|   Mom's Yeast Rolls|[2 cups hot water...|[2 cups hot water...|[2 cups hot water...|[cups hot water, ...|[water, margarine...|
|Sweet Potato Bread I|[1 1/2 cups white...|[1 1/2 cups white...|[1 1/2 cups white...|[cups white sugar...|[white sugar, veg...|
|         Orange Buns|[1/4 cup butter, ...|[1/4 cup butter, ...|[1/4 cup butter, ...|[cup butter, teas..

In [ ]:
with open("vegetarian.txt") as veg_file:
    veg_list = veg_file.read().splitlines()
print(veg_list)

test = allrecipes_df.withColumn("veg_label", udf_detectVege("ingredients"))
test = test.select("ingredients", "veg_label")
test.show(5)

# test_veg = test.filter(test.veg_label != "")
# test_veg.show(5)
# veg_pandas = test_veg.toPandas()

In [18]:
cleanIngredient_table = cleanIngredient_df.select("title", "rm_complete")
cleanIngredient_table = cleanIngredient_table.withColumn("Ing_Str", udf_convertString("rm_complete"))
# cleanIngredient_table = cleanIngredient_table.withColumn("Ing_Str", udf_convertString2("rm_complete"))
cleanIngredient_table = cleanIngredient_table.select("title", col('Ing_Str').alias('Ingredient'))
cleanIngredient_table = cleanIngredient_table.withColumn("ID", monotonically_increasing_id())

cleanIngredient_table.show(5)
cleanIngredient_table.count()

cleanIngredient_pandas = cleanIngredient_table.toPandas()
cleanIngredient_pandas.to_csv("sample_clean.csv")

# cleanIngredient_table.createOrReplaceTempView('df')
# cleanIngredient_table = spark.sql('select row_number() over (order by "ID") as num, * from df')
# cleanIngredient_table.show(5)
# cleanIngredient_table.count()


+--------------------+--------------------+---+
|               title|          Ingredient| ID|
+--------------------+--------------------+---+
|Basil, Roasted Pe...|unsalted-butter c...|  0|
|Crispy Cheese Twists|Parmesan-cheese g...|  1|
|   Mom's Yeast Rolls|water margarine w...|  2|
|Sweet Potato Bread I|white-sugar veget...|  3|
|         Orange Buns|butter white-suga...|  4|
+--------------------+--------------------+---+
only showing top 5 rows



In [12]:
tokenizer = Tokenizer(inputCol="Ingredient", outputCol="Component")
componentData = tokenizer.transform(cleanIngredient_table)
hashingTF = HashingTF(inputCol="Component", outputCol="Ing_Component")
featurizedData = hashingTF.transform(componentData)
idf = IDF(inputCol="Ing_Component", outputCol="Ing_Feature")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

normalizer = Normalizer(inputCol="Ing_Feature", outputCol="norm")
similarity = normalizer.transform(rescaledData)
sim_matrix = similarity.select("ID", "title", "norm")
sim_matrix.show()
sim_test = sim_matrix.rdd.map(lambda row:IndexedRow(row.ID, row.norm.toArray()))
type(sim_test)

# mat = IndexedRowMatrix(similarity.select("title", "norm")\
#                      .rdd.map(lambda row: IndexedRow(row.title, row.norm.toArray()))).toBlockMatrix()
# dot = mat.multiply(mat.transpose())
# dot.toLocalMatrix().toArray()

# dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())
# data.alias("i").join(data.alias("j"), col("i.title") < col("j.title"))\
#     .select(
#         col("i.title").alias("i"), 
#         col("j.title").alias("j"), 
#         dot_udf("i.norm", "j.norm").alias("dot"))\
#     .sort("i", "j")\
#     .show()


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:51636)
Traceback (most recent call last):
  File "c:\users\chest\appdata\local\programs\python\python37\lib\site-packages\py4j\java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\chest\appdata\local\programs\python\python37\lib\site-packages\py4j\java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:51636)

In [ ]:
sim_mat = IndexedRowMatrix(sim_test).toBlockMatrix()
dot = sim_mat.multiply(sim_mat.transpose())
dot.toLocalMatrix().toArray()
dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())

similarity.alias("i").join(similarity.alias("j"), col("i.ID") < col("j.ID"))\
    .select(
        col("i.ID").alias("i"), 
        col("j.ID").alias("j"), 
        dot_udf("i.ID", "j.ID").alias("cosine"))\
    .sort("i", "j")\
    .show()
